<a href="https://colab.research.google.com/github/gyuwonh-ai/SeSAC_miniProject_1Team/blob/main/YOLO_applied_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# 1. 드라이브 마운트 (이미 되어 있다면 생략 가능)
try:
    drive.mount('/content/drive')
except:
    pass

Mounted at /content/drive


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.3 MB/s eta 0:00:00


In [ ]:
import cv2
from ultralytics import YOLO
import os
import time


# ==========================================
# [설정] 경로를 본인 환경에 맞게 수정하세요
# ==========================================
# 입력 파일 경로 (분석할 원본 영상)
input_video_path = '/content/drive/MyDrive/shared_googledrive(Sessac Final Project)/dataset/Mermaid/필라테스_가산_A_Mat_Mermaid_고급_actorP093_20221014_11.58.52_CAM_1.mp4'

# 출력 파일 경로 (결과가 저장될 위치)
output_video_path = '/content/drive/MyDrive/shared_googledrive(Sessac Final Project)/video/yolo_mermaid.mp4'

# 모델 경로 (x 모델 사용)
model_path = 'yolo11x-pose.pt'
# ==========================================

def process_video(input_path, output_path, model_path):
    # 1. 모델 로드
    print(f"⏳ 모델 로딩 중... ({model_path})")
    model = YOLO(model_path)

    # 2. 비디오 캡처 초기화
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"❌ 영상을 열 수 없습니다: {input_path}")
        return

    # 3. 비디오 속성 가져오기 (저장용)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"ℹ️ 영상 정보: {width}x{height}, {fps:.2f} FPS, 총 {total_frames} 프레임")

    # 4. 비디오 저장 설정 (VideoWriter)
    # 코덱: 'mp4v'가 일반적입니다. (Colab/Web 호환성 좋음)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_idx = 0
    start_time = time.time()

    print("🚀 변환 시작! (시간이 좀 걸릴 수 있습니다)")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break # 영상 끝

        # 5. YOLO 추론 (Inference)
        # conf=0.5: 확신할 수 있는 관절만 그리기 (조절 가능)
        results = model(frame, verbose=False, conf=0.5)

        # 6. 결과 시각화 (Plotting)
        # results[0].plot()은 원본 프레임 위에 스켈레톤을 예쁘게 그려서 반환해줍니다.
        annotated_frame = results[0].plot(img=frame)

        # (선택사항) 화면에 진행률 표시 텍스트 추가
        cv2.putText(annotated_frame, f"Frame: {frame_idx}/{total_frames}", (20, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # 7. 프레임 저장
        out.write(annotated_frame)

        frame_idx += 1

        # 진행 상황 출력 (50프레임마다)
        if frame_idx % 50 == 0:
            progress = (frame_idx / total_frames) * 100
            print(f"   Processing: {progress:.1f}% ({frame_idx}/{total_frames})", end='\r')

    # 8. 자원 해제
    cap.release()
    out.release()

    elapsed = time.time() - start_time
    print(f"\n✅ 변환 완료! 저장 위치: {output_path}")
    print(f"⏱️ 소요 시간: {elapsed:.1f}초")

# 실행
if __name__ == "__main__":
    # 파일이 실제 존재하는지 체크 후 실행
    if os.path.exists(input_video_path):
        process_video(input_video_path, output_video_path, model_path)
    else:
        print(f"❌ 입력 파일을 찾을 수 없습니다: {input_video_path}")
        print("   경로를 다시 확인해주세요.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
⏳ 모델 로딩 중... (yolo11x-pose.pt)
ℹ️ 영상 정보: 2448x2048, 30.00 FPS, 총 819 프레임
🚀 변환 시작! (시간이 좀 걸릴 수 있습니다)

✅ 변환 완료! 저장 위치: /content/drive/MyDrive/shared_googledrive(Sessac Final Project)/video/yolo_mermaid.mp4
⏱️ 소요 시간: 107.7초
